Webscraping


In [162]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup
#test coucouc

In [163]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = "C:/Users/User/Documents/GitHub/Films_DM_2023/chromedriver_win32" #the path
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

C:\Users\User\AppData\Local\Temp\ipykernel_28192\1295019404.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [164]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)


In [165]:
#Aller dans films 
films = driver.find_element(By.XPATH, '//*[@title="Cinéma"]').click()
time.sleep(2)

In [167]:
#Aller dans films 
tous_films = driver.find_element(By.XPATH, '//*[@title="Tous les films"]').click()
time.sleep(2)


In [129]:
# Assuming you already have the driver set up
# driver = webdriver.Chrome(executable_path='path/to/chromedriver')

# Corrigé par Adin -> Il faudrait créer une loop qui modifie le numéro ce qui marche
#wait = WebDriverWait(driver, 20)
#action = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@href="/films/genre-13027/"]')))
#action.click()

In [130]:
#Choisir les genres
#action =driver.find_element(By.XPATH,'//*[@class="filter-entity-select"]/div[1]/select/option[2]').click()

#element =each_fiche.execute_script("return document.evaluate('//div[@class=\"meta-body-item meta-body-info\"]/text()[3]', document).iterateNext().textContent;")
    #/span/following-sibling::text()[1]
    #element= each_fiche.find_element(By.CLASS_NAME, 'spacer')
    #duree =element.find_element(By.XPATH, "//text()[1]").text
    #duree= element.find_element(By.XPATH,("following-sibling::*[2]")).text #The following sibling2 makes that we get the date and not the /
    #//*[@id="content-layout"]/section[3]/div[2]/ul/li[1]/div/div[1]/div[1]/div[1]/text()[1]

In [174]:
#Avoir feuille par feuille
films =[]
films_addedinfo=[]
links=[]
links_real=[]
links_actor1=[]
links_actor2=[]
real_info=[]
act1_info=[]
act2_info=[]
for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
    #titres
    titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
    #date sortie
    date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
    #durée
    duree =each_fiche.find_element(By.CLASS_NAME, "meta-body-info").text.split('/')[1]
    #Genres
    info =each_fiche.find_element(By.CLASS_NAME, "meta-body-info")
    genres= info.find_elements(By.CLASS_NAME, "xXx")
    #genre1
    genre_1 = genres[0].text
    #Genre 2 et 3
    if len(genres)>=2:
        genre_2= genres[1].text
        if len(genres)==3:
            genre_3=genres[2].text
        else:
            genre_3="NA"
    else:
        genre_2="NA"
        
    #Directors
    #realisateur= each_fiche.find_element(By.CLASS_NAME, "blue-link").text
    direction =each_fiche.find_element(By.CLASS_NAME, "meta-body-direction")
    realisateurs= direction.find_elements(By.CSS_SELECTOR, "a")
    
    #Get the link to the first realisator page to get info if he got awards later
    link_real= realisateurs[0].get_attribute('href')
    links_real.append(link_real)
    
    #realisateur1
    real_1 = realisateurs[0].text
    #real 2
    if len(realisateurs)>=2:
        real_2= realisateurs[1].text
    else:
        real_2="NA"
    
    #Actors
    casting =each_fiche.find_element(By.CLASS_NAME, "meta-body-actor")
    actors= casting.find_elements(By.CSS_SELECTOR, "a")
    
    #Get the link to the actors 1
    link_actor1= actors[0].get_attribute('href')
    links_actor1.append(link_actor1)
    
    #Get the link to the actors 2
    link_actor2= actors[1].get_attribute('href')
    links_actor2.append(link_actor2)
    
    #actor1
    actor_1 = actors[0].text
    #real 2
    if len(actors)>=2:
        actor_2= actors[1].text
        if len(actors)==3:
            actor_3=actors[2].text
        else:
            actor_3="NA"
    else:
        actor_2="NA"
        
    #Ratings
    #First initiate them in NA if does not exist
    note_spectateur="NA"
    note_presse="NA"
    #Look for the notes
    rating = each_fiche.find_element(By.CLASS_NAME, "rating-holder")
    ratings=rating.find_elements(By.CLASS_NAME,'rating-item-content')
    #For the first one
    rating_type_1=ratings[0].find_element(By.CSS_SELECTOR,'a').text
    if(rating_type_1== "PRESSE"):
        note_presse= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text
    elif(rating_type_1== "SPECTATEURS"):
        note_spectateur= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text   
        
    #For the second one
    rating_type_2=ratings[1].find_element(By.CSS_SELECTOR,'a').text
    if(rating_type_2== "SPECTATEURS"):
        note_spectateur= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text 
    elif(rating_type_2== "PRESSE"): #sometimes it is not in the same order
        note_presse= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text
    
    #Now we are done with the information we could find on the first page => click on the link for each film and go see more info
    #Problem, if come back to main page, stale elements => work around it by first creating a vector with all the links and visiting at the end without coming back to the main page
    link= each_fiche.find_element(By.CLASS_NAME, "meta-title-link").get_attribute('href')
    links.append(link)
    

    films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "duree": duree,
                "genre1": genre_1,
                "genre2": genre_2,
                "genre3": genre_3,
                "réalisateur_1": real_1,
                "réalisateur_2": real_2,
                "actor_1": actor_1,
                "actor 2": actor_2,
                "actor 3": actor_3,
                "note_presse": note_presse,
                "note spectators": note_spectateur,
                "link":link,
                "link realisator": link_real,
            })
    
#Get info on second page (more info on films)
for link in links:
    driver.get(link)
    time.sleep(2)
    infos = driver.find_element(By.CLASS_NAME,'meta-body')
    #Scenaristes
    scenar = infos.find_element(By.XPATH,'div[3]')
    scenarists = scenar.find_elements(By.CSS_SELECTOR,'a')
    scenar_1 = scenarists[0].text
    
    #Technical informations
    techni= driver.find_element(By.CLASS_NAME, 'ovw-technical')
    #nationality
    natio=techni.find_element(By.CLASS_NAME, 'nationality').text
    
    #Distributor
    nom=techni.find_element(By.XPATH,'div[3]/span').text
    if(nom == "Distributeur"):
        if techni.find_elements(By.XPATH,'div[3]/a'):
            distributor = techni.find_element(By.XPATH,'div[3]/a').text
        else: 
            distributor="NA"
            
    #Palmares
    
    #Attention, this does not exist for all the films!!!
    
    #If it exists
    if(techni.find_element(By.XPATH,'div[4]/span').text == "Récompense" or techni.find_element(By.XPATH,'div[4]/span').text == "Récompenses" ):
        #if techni.find_element(By.XPATH,'div[4]/a'):
        recompense = techni.find_element(By.XPATH,'div[4]/a').text
        recompense=str(re.sub("[^0-9^]","",recompense))
        #As if there is a recompense, there is one row more in the table with infos, it changes our div => put for the ones with recompenses here
        budget=0
        
        #Box office
        exist=techni.find_element(By.XPATH,'div[11]/span').text
        if(exist == "Box Office France"):
            boxeoffice=techni.find_element(By.XPATH,'div[11]/a').text
        elif (exist == "Budget"): #As there is not always a raw boxoffice
            budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
        #Budget
        exist2=techni.find_element(By.XPATH,'div[12]/span').text
        if(exist2 == "Budget"):
            budget=techni.find_element(By.XPATH,'div[12]/span[2]').text
        elif (exist2 == "Langues"): #As there is not always a raw boxoffice and this repercutes
            language=techni.find_element(By.XPATH,'div[12]/span[2]').text
            
        #Language
        exist6=techni.find_element(By.XPATH,'div[13]/span').text
        if(exist6 == "Langues"):
            language=techni.find_element(By.XPATH,'div[13]/span[2]').text
        
        
    else:  #If the film has no recompense field 
        recompense= "0"
        
        #Budget
        budget="NA"
        boxeoffice="NA"
    
        #Box office
        exist3=techni.find_element(By.XPATH,'div[10]/span').text
        if(exist3 == "Box Office France"):
            boxeoffice=techni.find_element(By.XPATH,'div[10]/a').text
        elif (exist3 == "Budget"):
            budget=techni.find_element(By.XPATH,'div[10]/span[2]').text
            
        #Budget
        exist4=techni.find_element(By.XPATH,'div[11]/span').text
        if(exist4 == "Budget"):
            budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
        elif (exist4 == "Langues"):
            language=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
        #Language
        exist5=techni.find_element(By.XPATH,'div[12]/span').text
        if(exist5 == "Langues"):
            language=techni.find_element(By.XPATH,'div[12]/span[2]').text
  

    films_addedinfo.append({
                "Scenar": scenar_1,
                "nationality": natio,
                "distributor": distributor,
                "nominations": recompense,
                "budget": budget,
                "language": language,
                "box-office":boxeoffice,
            })
    
    
#Get more info on the realisator

for real in links_real:
    driver.get(real)
    time.sleep(2)
    real_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
    if real_infos.find_elements(By.CSS_SELECTOR,'a'):
        award_real = real_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
    else: 
        award_real=0
    real_info.append({"real awards":award_real,})
    
    
#Get more info on the actors 1
for actor1 in links_actor1:
    driver.get(actor1)
    time.sleep(2)
    if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
        act1_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
        if act1_infos.find_elements(By.CSS_SELECTOR,'a'):
            award_act1 = act1_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
        else: 
            award_act1=0
    else: 
        award_act1="NA"
    act1_info.append({"first actor awards":award_act1,})

#Get more info on the actors 2
for actor2 in links_actor2:
    driver.get(actor2)
    time.sleep(1.5)
    if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
        act2_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
        if act2_infos.find_elements(By.CSS_SELECTOR,'a'):
            award_act2 = act2_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
        else: 
            award_act2=0
    else:
        award_act2="NA"
    act2_info.append({"Second actor awards":award_act2,})

            
    
#dataframe
films_data = pd.DataFrame(films)
films_added= pd.DataFrame(films_addedinfo)
real_info=pd.DataFrame(real_info)
act1_info=pd.DataFrame(act1_info)
act2_info=pd.DataFrame(act2_info)
Movies = pd.concat([films_data, films_added,real_info,act1_info,act2_info], axis=1)
Movies

,Titre,date_sortie,duree,genre1,genre2,genre3,réalisateur_1,réalisateur_2,actor_1,actor 2,actor 3,note_presse,note spectators,link,link realisator,Scenar,nationality,distributor,nominations,budget,language,box-office,real awards,first actor awards,Second actor awards
0,Les Gardiens de la Galaxie 3,3 mai 2023,2h 30min,Action,Fantastique,Science Fiction,James Gunn,NA,Chris Pratt,Zoe Saldana,Dave Bautista,"3,6","4,3",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,James Gunn,U.S.A.,The Walt Disney Company France,0,-,Anglais,1 938 600 entrées,1,0,0
1,Jeanne du Barry,17 mai 2023,1h 56min,Drame,Historique,Romance,Maïwenn,NA,Maïwenn,Johnny Depp,Benjamin Lavernhe,"3,1","3,6",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Maïwenn,France,Le Pacte,1,-,Français,45 029 entrées,13,13,9
2,Les Trois Mousquetaires: D'Artagnan,5 avril 2023,2h 01min,Aventure,Historique,NA,Martin Bourboulon,NA,François Civil,Vincent Cassel,Romain Duris,"3,7","4,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Alexandre De La Patellière,France,Pathé,0,60 M€,Français,3 075 068 entrées,1,1,7
3,Je verrai toujours vos visages,29 mars 2023,1h 58min,Drame,NA,NA,Jeanne Herry,NA,Adèle Exarchopoulos,Dali Benssalah,Leïla Bekhti,"4,1","4,4",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Jeanne Herry,France,StudioCanal,0,-,Français,1 060 802 entrées,2,6,0
4,"Super Mario Bros, le film",5 avril 2023,1h 32min,Animation,Comédie,Famille,Aaron Horvath,Michael Jelenic,Pierre Tessier,Chris Pratt,Jérémie Covillault,"3,1","4,2",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Matthew Fogel,U.S.A.,Universal Pictures International France,0,-,Anglais,6 628 418 entrées,0,0,0
5,La Vie pour de vrai,19 avril 2023,1h 50min,Comédie,NA,Famille,Dany Boon,NA,Dany Boon,Charlotte Gainsbourg,Kad Merad,"2,8","2,7",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Dany Boon,France,Pathé,0,-,Français,763 181 entrées,8,8,4
6,AKA,28 avril 2023,2h 00min,Action,Policier,Thriller,Morgan S. Dalibert,NA,Alban Lenoir,Eric Cantona,Thibault de Montalembert,"3,3","3,5",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Morgan S. Dalibert,France,NA,0,-,Français,NA,1,1,1
7,Misanthrope,26 avril 2023,1h 58min,Policier,Thriller,NA,Damián Szifron,NA,Shailene Woodley,Ben Mendelsohn,Jovan Adepo,"3,7","3,8",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Damián Szifron,U.S.A.,Metropolitan FilmExport,0,-,Anglais,169 707 entrées,2,3,5
8,Fast & Furious X,17 mai 2023,2h 21min,Action,NA,NA,Louis Leterrier,NA,Vin Diesel,Michelle Rodriguez,Jason Momoa,"2,9","3,5",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Dan Mazeau,U.S.A.,Universal Pictures International France,0,-,Anglais,NA,2,2,1
9,Burning Days,26 avril 2023,2h 08min,Drame,Thriller,NA,Emin Alper,NA,Selahattin Paşalı,Ekin Koç,Erol Babaoğlu,"3,7","3,9",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Emin Alper,Turquie,Memento Distribution,6,-,Turc,95 865 entrées,2,NA,0
